In [11]:
!nvidia-smi

Thu Oct 23 12:51:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
import torch
from torch import nn
torch.device('cpu'),torch.device('cuda'),torch.device('cuda:1')
#torch.device 是 PyTorch 中用于表示计算设备（如 CPU 或 GPU）的类，它的主要作用是明确指定张量（Tensor）或模型（Module）在哪个设备上进行计算

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [13]:
torch.cuda.device_count()

1

In [14]:
def try_gpu(i=0):
  """如果存在，则返回gpu(i),否则返回cpu()"""
  if torch.cuda.device_count() >= i + 1:
    #torch.cuda.device_count()返回当前系统中可用的 GPU 数量（整数）。若返回 0，表示无可用 GPU。
    #torch.cuda 是 PyTorch 中用于管理和操作 NVIDIA GPU 的模块，提供了一系列与 GPU 相关的功能
    return torch.device(f'cuda:{i}')
  return torch.device('cpu')

def try_all_gpus():
  """返回所有可用的GPU,如果没有GPU，则返回[cpu(),]"""
  devices = [torch.device(f'cuda:{i}')  for i in range(torch.cuda.device_count())]
  return devices if devices else [torch.device('cpu')]

try_gpu(),try_gpu(10),try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [15]:
#张量与GPU
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

In [16]:
#存储在GPU上
X = torch.ones(2,3,device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [23]:
#复制
Y = torch.tensor([1,2,3])
Z = Y.cuda(0)
print(X)
print(Z)
print(Y)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([1, 2, 3], device='cuda:0')
tensor([1, 2, 3])


In [24]:
X+Z

tensor([[2., 3., 4.],
        [2., 3., 4.]], device='cuda:0')

In [28]:
Z.cuda(0) is Z
#变量已经位于第一个GPU上，如果我们还调用Z.cuda(0),它将返回Z,而不会复制并分配新内存

True

In [31]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())
#net.to(...) 是 PyTorch 中模型（nn.Module）的方法，用于将模型的所有参数（权重、偏置等）转移到指定的计算设备（CPU 或 GPU）。
#device=try_gpu() 中，try_gpu() 是之前定义的函数（作用是：若存在 GPU 则返回 GPU 设备，否则返回 CPU 设备），这里作为参数指定目标设备。
#赋值 net = ... 表示将转移到新设备后的模型重新赋值给变量 net（原模型的参数仍在原来的设备上，新模型是转移后的版本）。

#net.to(...) 中的 net 只是变量名，其作用取决于这个变量指向的对象是否是 PyTorch 的模型（nn.Module 及其子类，如 nn.Sequential、自定义模型等）。
#变量名本身不影响 to(...) 方法的功能，只要变量指向的是模型对象，to(...) 就有效

#to(...) 是 PyTorch 中所有神经网络模型（继承自 nn.Module）自带的方法，用于将模型参数转移到指定设备（CPU/GPU）。
#无论变量名叫什么（net、model、my_network 等），只要这个变量指向一个模型对象，就可以调用 to(...) 方法。

## 定义模型，变量名叫 model（不是 net）
# model = nn.Sequential(nn.Linear(3, 1))

# 调用 to(...) 方法，完全有效
# model = model.to(device=try_gpu())  # 将模型转移到目标设备



In [32]:
net(X)

tensor([[0.5686],
        [0.5686]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [33]:
#我们确定模型参数储存在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)